<a href="https://colab.research.google.com/github/Anonymous143w/ML-Projects/blob/main/pbl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
        

In [137]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [5]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [6]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'sad', 'angry']

In [19]:
!pip install -q kaggle

In [21]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"whysoserious143","key":"09687819c898715470d5b9d97e79397d"}'}

In [22]:
! mkdir ~/.kaggle

In [23]:
!cp kaggle.json ~/.kaggle/

In [24]:
!chmod 600 ~/.kaggle/kaggle.json

In [26]:
!kaggle dataset list

usage: kaggle
       [-h]
       [-v]
       {competitions,c,datasets,d,kernels,k,config}
       ...
kaggle: error: argument command: invalid choice: 'dataset' (choose from 'competitions', 'c', 'datasets', 'd', 'kernels', 'k', 'config')


In [25]:
!kaggle datasets download -d ejlok1/toronto-emotional-speech-set-tess

100% 426M/428M [00:18<00:00, 18.2MB/s]
100% 428M/428M [00:18<00:00, 24.5MB/s]


In [146]:
!unzip toronto-emotional-speech-set-tess.zip

In [147]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/A/Actor_*//*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [148]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)


In [149]:
#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(552, 184)


In [150]:
#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')


Features extracted: 180


In [163]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=64, epsilon=1e-08, hidden_layer_sizes=(400,), learning_rate='adaptive', max_iter=750)

In [164]:
#DataFlair - Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=64, hidden_layer_sizes=(400,),
              learning_rate='adaptive', max_iter=750)

In [165]:
#DataFlair - Predict for the test set
y_pred=model.predict(x_test)

In [166]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 76.09%


In [93]:
from sklearn.metrics import accuracy_score, f1_score

In [94]:
f1_score(y_test, y_pred,average=None)

array([0.73846154, 0.73417722, 0.69230769, 0.75      ])

In [95]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,sad,sad
1,calm,calm
2,calm,happy
3,sad,sad
4,happy,happy
5,calm,sad
6,happy,happy
7,happy,happy
8,calm,calm
9,sad,sad


In [96]:
import pickle
# Writing different model files to file
with open( 'PBL.sav', 'wb') as f:
    pickle.dump(model,f)

In [98]:
filename = 'PBL.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/A/audio_2023-04-28_11-51-59.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['happy'], dtype='<U5')

In [99]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
